# Basic Gates

Basic quantum gates.

The bloqs in this module encode gates you'd expect to find in any quantum computing
framework. It includes single-qubit unitary gates like rotations, bit- and phase-flip;
basic multi-qubit unitary gates; states and effects in the Pauli basis; and non-Clifford
gates `TGate` and `Toffoli` which are commonly counted to estimate algorithm resource
requirements.

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `CNOT`
Two-qubit controlled-NOT.

#### Registers
 - `ctrl`: One-bit control register.
 - `target`: One-bit target register.


In [ ]:
from qualtran.bloqs.basic_gates import CNOT

bloq = CNOT()
show_bloq(bloq)

We show how CNOT is represented as a facorized tensor during Quimb tensor simulation.

In [ ]:
from qualtran.simulation.tensor import (
    cbloq_to_quimb, get_right_and_left_inds
)

cbloq = bloq.as_composite_bloq()
tn, _ = cbloq_to_quimb(cbloq)

# Rename the indices to something less verbose
from qualtran._infra.composite_bloq import _get_dangling_soquets
lsoqs = _get_dangling_soquets(cbloq.signature, right=False)
rsoqs = _get_dangling_soquets(cbloq.signature, right=True)

# Note: the following only works for single-dimension soquets.
rename = {lsoqs[k]: f'{k}_in' for k in lsoqs.keys()}
rename |= {rsoqs[k]: f'{k}_out' for k in rsoqs.keys()}
tn = tn.reindex(rename)

tn.draw(color=['COPY', 'XOR'], show_tags=False, initial_layout='spectral')

In [ ]:
bloq.tensor_contract()

## `PlusState`
The state |+>

In [ ]:
from qualtran.bloqs.basic_gates import PlusState

bloq = PlusState()
show_bloq(bloq)

In [ ]:
bloq.tensor_contract()

## `ZeroState`
The state |0>

In [ ]:
from qualtran.bloqs.basic_gates import ZeroState

bloq = ZeroState()
show_bloq(bloq)

## Synthesizing a Bell state

We can create a bell state (|00> + |11>) by cnotting a |+> state into a |0> state.

In [ ]:
from qualtran.bloqs.basic_gates import ZeroState

bb = BloqBuilder()

q0 = bb.add(PlusState())
q1 = bb.add(ZeroState())

q0, q1 = bb.add(CNOT(), ctrl=q0, target=q1)

cbloq = bb.finalize(q0=q0, q1=q1)
show_bloq(cbloq)

In [ ]:
cbloq.tensor_contract()

We can see how this composite bloq corresponds to a quimb tensor network.

In [ ]:
tn, _ = cbloq_to_quimb(cbloq)
lsoqs = _get_dangling_soquets(cbloq.signature, right=False)
rsoqs = _get_dangling_soquets(cbloq.signature, right=True)

# Note: the following only works for single-dimension soquets.
rename = {lsoqs[k]: f'{k}_in' for k in lsoqs.keys()}
rename |= {rsoqs[k]: f'{k}_out' for k in rsoqs.keys()}
tn = tn.reindex(rename)

tn.draw(color=['COPY', 'XOR', '0', '+'], show_tags=False, initial_layout='spectral')

`PlusState` and `ZeroState` provide `as_cirq_op` methods for Cirq interoperability, resulting in the familiar `cirq.Circuit` for constructing a Bell state.

In [ ]:
from cirq.contrib.svg import SVGCircuit
circuit, qubits = cbloq.to_cirq_circuit()
SVGCircuit(circuit)

## `Rz`
Single-qubit Rz gate.

#### Parameters
 - `angle`: Rotation angle in radians.
 - `eps`: precision for implementation of rotation. 

#### Registers
 - `q`: One-bit register. 

#### References
[Efficient synthesis of universal Repeat-Until-Success
circuits](https://arxiv.org/abs/1404.5320), which offers a small improvement
[Optimal ancilla-free Clifford+T approximation
of z-rotations](https://arxiv.org/pdf/1403.2975.pdf).


In [ ]:
from qualtran.bloqs.basic_gates import Rz

bloq = Rz(angle=np.pi / 4.0)
show_bloq(bloq)

## `Hadamard`
The Hadamard gate

This converts between the X and Z basis.

$$
H |0\rangle = |+\rangle \\
H |-\rangle = |1\rangle
$$

#### Registers
 - `q`: The qubit


In [ ]:
from qualtran.bloqs.basic_gates import Hadamard

bloq = Hadamard()
show_bloq(bloq)